In [72]:
# Import required libraries and dependencies
import numpy as np
import pandas as pd
import hvplot.pandas
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import kneed
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [3]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [4]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [5]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
market_data_scaled = StandardScaler().fit_transform(df_market_data)

In [6]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(
    market_data_scaled,
    columns=df_market_data.columns
)
# Inspect DataFrame
df_market_data_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
0,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
1,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
2,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
3,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
4,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


In [7]:

# Copy the crypto names from the original data
df_market_data_scaled["coin_id"] = df_market_data.index

# Set the coinid column as index
df_market_data_scaled = df_market_data_scaled.set_index("coin_id")

# Display sample data
df_market_data_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Data.

In [17]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 11))
# Print the list of k-values
k_values

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [73]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for k in k_values:
    model = KMeans(n_clusters=k, random_state=0)
    model.fit(df_market_data_scaled)
    inertia.append(model.inertia_)

In [19]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k_values, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame(elbow_data)

# Inspect the DataFrame
elbow_df

,k,inertia
0,1,287.000000
1,2,195.820218
2,3,123.190482
3,4,79.022435
4,5,65.302379
5,6,52.888518
6,7,43.914690
7,8,37.517032
8,9,32.485241
9,10,28.222899


In [66]:
# Create legend labels for the sake of the ultimate composite elbow plot
legend_labels = ["Original Elbow Curve", "PCA Elbow Curve"]

In [67]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.

elbow_hvplot = elbow_df.hvplot.line(
    x = "k",
    y = "inertia",
    title = "Elbow Curve",
    xticks = k,
    color = "blue",
    label = legend_labels[0]
)
elbow_hvplot

:Curve   [k]   (inertia)

In [24]:
# Use kneed module to identify the elbow point programmatically
kl = kneed.KneeLocator(
    range(1, 11), inertia, curve="convex", direction="decreasing"
)
print(f"The optimal number of clusters is: {kl.elbow}")

The optimal number of clusters is: 4


#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** Based on analysis of the second derivative of the elbow curve, the best value for k is 4. Visually, we can see the elbow at k=4, and kneed's KneeLocator confirms this.

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [25]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=kl.elbow)

In [26]:
# Fit the K-Means model using the scaled data
model.fit(df_market_data_scaled)

c:\Users\e.a.wright\anaconda3\envs\datascience\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\e.a.wright\anaconda3\envs\datascience\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=4)

In [27]:
# Predict the clusters to group the cryptocurrencies using the scaled data
crypto_clusters = model.predict(df_market_data_scaled)

# Print the resulting array of cluster values.
print(crypto_clusters)

[0 0 1 1 0 0 0 0 0 1 1 1 1 0 1 0 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 2 0 1 1 3
 1 1 1 1]


In [28]:
# Create a copy of the DataFrame
copy_crypto_df = df_market_data_scaled.copy()

In [75]:
# Add a new column to the DataFrame with the predicted clusters
copy_crypto_df["crypto_clusters"] = crypto_clusters

# Temporarily change the crypto_clusters column to a string type using the astype() method
copy_crypto_df["crypto_clusters"] = copy_crypto_df["crypto_clusters"].astype(str)

# Display sample data
copy_crypto_df.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,crypto_clusters
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,0
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,0
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,1
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,1
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,0


In [76]:
copy_crypto_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41 entries, bitcoin to digibyte
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   price_change_percentage_24h   41 non-null     float64
 1   price_change_percentage_7d    41 non-null     float64
 2   price_change_percentage_14d   41 non-null     float64
 3   price_change_percentage_30d   41 non-null     float64
 4   price_change_percentage_60d   41 non-null     float64
 5   price_change_percentage_200d  41 non-null     float64
 6   price_change_percentage_1y    41 non-null     float64
 7   crypto_clusters               41 non-null     object 
dtypes: float64(7), object(1)
memory usage: 2.9+ KB


In [77]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
crypto_clusters_hvplot = copy_crypto_df.hvplot.scatter(
    x = "price_change_percentage_24h",
    y = "price_change_percentage_7d",
    title = "Crypto Clusters Plotted by Price Change Percentage (24h vs. 7d)",
    color = "crypto_clusters",
    hover_cols = ["coin_id"]
)
crypto_clusters_hvplot

:Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,crypto_clusters,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [31]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [32]:
# Use the PCA model with `fit_transform` to reduce to three principal components.
crypto_pca_data = pca.fit_transform(df_market_data_scaled)

# View the first five rows of the DataFrame. 
crypto_pca_data[:5]

array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527]])

In [33]:
# Retrieve the explained variance to determine how much information can be attributed to each principal component.
pca.explained_variance_ratio_

array([0.3719856 , 0.34700813, 0.17603793])

In [35]:
# Find the total explained variance by adding up all the explained variance ratios.
total_explained_variance = pca.explained_variance_ratio_.sum()
total_explained_variance

0.8950316570309841

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** The total explained variance of the three principal components is 0.895.

In [36]:
# Create a new DataFrame with the PCA data.
crypto_pca_df = pd.DataFrame(crypto_pca_data, columns=["PC1", "PC2", "PC3"])

# Copy the crypto names from the original data
crypto_pca_df["coin_df"] = df_market_data.index

# Set the coinid column as index
crypto_pca_df = crypto_pca_df.set_index("coin_df")

# Display sample data
crypto_pca_df.head()

,PC1,PC2,PC3
coin_df,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


---

### Find the Best Value for k Using the PCA Data

In [37]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 11))

In [74]:
# Create an empty list to store the inertia values
pca_inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for k in k_values:
    model = KMeans(n_clusters=k, random_state=0)
    model.fit(crypto_pca_df)
    pca_inertia.append(model.inertia_)

In [43]:
# Create a dictionary with the data to plot the Elbow curve
pca_elbow_data = {"k": k_values, "inertia": pca_interia}

# Create a DataFrame with the data to plot the Elbow curve
pca_elbow_df = pd.DataFrame(pca_elbow_data)

In [68]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
pca_elbow_hvplot = pca_elbow_df.hvplot.line(
    x = "k",
    y = "inertia",
    title = "Cryptocurrency Price Change PCA Elbow Curve",
    color = "red",
    label = legend_labels[1]
)
pca_elbow_hvplot

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** The best value for k when using the PCA data is 4.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** It does not differ from the best k value found using the original data.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [46]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=pca_kl.elbow)

In [47]:
# Fit the K-Means model using the PCA data
model.fit(crypto_pca_df)

c:\Users\e.a.wright\anaconda3\envs\datascience\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\e.a.wright\anaconda3\envs\datascience\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=4)

In [48]:
# Predict the clusters to group the cryptocurrencies using the PCA data
pca_stock_clusters = model.predict(crypto_pca_df)

# Print the resulting array of cluster values.
print(pca_stock_clusters)

[3 3 0 0 3 3 3 3 3 0 0 0 0 3 0 3 0 0 3 0 0 3 0 0 0 0 0 0 3 0 0 0 1 3 0 0 2
 0 0 0 0]


In [78]:
# Create a copy of the DataFrame with the PCA data
copy_pca_crypto_df = crypto_pca_df.copy()

# Add a new column to the DataFrame with the predicted clusters
copy_pca_crypto_df["pca_crypto_clusters"] = pca_stock_clusters

# Temporarily change the crypto_clusters column to a string type using the astype() method
copy_pca_crypto_df["pca_crypto_clusters"] = copy_pca_crypto_df["pca_crypto_clusters"].astype(str)

# Display sample data
copy_pca_crypto_df.head()

,PC1,PC2,PC3,pca_crypto_clusters
coin_df,,,,
bitcoin,-0.600667,0.842760,0.461595,3
ethereum,-0.458261,0.458466,0.952877,3
tether,-0.433070,-0.168126,-0.641752,0
ripple,-0.471835,-0.222660,-0.479053,0
bitcoin-cash,-1.157800,2.041209,1.859715,3


In [79]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
pca_cypto_hvplot = copy_pca_crypto_df.hvplot.scatter(
    x = "PC1",
    y = "PC2",
    title = "Cryptocurrency Clusters Plotted by PCA",
    # Set color to the pca_crypto_clusters
    color = "pca_crypto_clusters",
    # Change color scale to categorical and options to 1, 2, 3
    cmap = "Set1",
    hover_cols = ["coin_df"]
)
pca_cypto_hvplot

:Scatter   [PC1]   (PC2,pca_crypto_clusters,coin_df)

In [80]:
# Now, for exploratory purposes, create a 3d scatter plot using plotly express
pca_3d_hvplot = px.scatter_3d(
    x = copy_pca_crypto_df["PC1"],
    y = copy_pca_crypto_df["PC2"],
    z = copy_pca_crypto_df["PC3"],
    title = "Cryptocurrency Clusters Plotted by PCA",
    color = copy_pca_crypto_df["pca_crypto_clusters"],
    hover_name = copy_pca_crypto_df.index,
    width = 600,
    height = 600
)

# Update the layout to set the axis labels
pca_3d_hvplot.update_layout(
    scene=dict(
        xaxis_title="PC1",
        yaxis_title="PC2",
        zaxis_title="PC3"
    )
)

pca_3d_hvplot

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [69]:
# Composite plot to contrast the Elbow curves

composite_plot = elbow_hvplot * pca_elbow_hvplot
composite_plot = composite_plot.opts(legend_position="top_right")

# Display the composite plot
composite_plot

:Overlay
   .Curve.Original_Elbow_Curve :Curve   [k]   (inertia)
   .Curve.PCA_Elbow_Curve      :Curve   [k]   (inertia)

In [70]:
# Calculate a mean difference in inertia between the two methods
mean_diff_inertia = np.mean(pca_inertia) - np.mean(inertia)

# Print the mean difference in inertia between the two methods with text
print(f"The mean difference in inertia between the two methods is {mean_diff_inertia:.2f}.")

The mean difference in inertia between the two methods is -25.09.


#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** The impact of using fewer features to cluster the data using K-Means is that, for all values of k, the clusters are more tightly grouped around the centroids. The mean difference in inertia between the original data and the PCA data is -25.09. However, there is no difference in the best value for k. Both elbows are at k=4.